In [107]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import datetime as dt
import seaborn as sns
from matplotlib import style

import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
color_pal = sns.color_palette()
plt.style.use('fivethirtyeight')

import random

In [108]:
#LP MAXIMIZATION MODEL

from pulp import *
Promo_Model = LpProblem("Pharma_Promotion_Model",LpMaximize)

In [109]:
#PARAMETERS

T=10 # end of promotion horizon
PHorizon = list(range(0, T))

I=3 # product count 
Products = list(range(0, I))

#NOT USED RIGHT NOW
Promo_Levels = [0,1,2]

for i in Products:
    for t in PHorizon:
       Demand[i][t] = random.randint(0, 10000)
    
Total_PaidQty_Limit = 10000
Total_FreeQty_Limit = 2000

In [110]:
#DECISION VARIABLES

#Paid Quantity
PaidQtyLimit = 2000
P = LpVariable.dicts("Paid Quantity",(Products,PHorizon),lowBound=0, upBound=PaidQtyLimit, cat='Integer')

#Free Quantity
FreeQtyLimit = 500
F = LpVariable.dicts("Free Quantity",(Products,PHorizon),lowBound=0, upBound=FreeQtyLimit, cat='Integer')

#Promo Ratio
R = LpVariable.dicts("Promo Ratio",(Products,PHorizon),lowBound=0, upBound=1, cat='Continuous')

#Is Product "i" at Time "t" Promoted?
X = LpVariable.dicts("IsPromoted",(Products,PHorizon),cat='Binary')

In [111]:
#OBJECTIVE FUNCTION

Total_Paid_Quantity = lpSum(lpSum(P[i][t] for i in Products) for t in PHorizon) 

Promo_Model += Total_Paid_Quantity

In [115]:
#DEMAND SATISFACTION
for i in Products:
    for t in PHorizon:
       Promo_Model += (P[i][t] + F[i][t]) == Demand[i][t]

#IS PROMOTED
for i in Products:
    for t in PHorizon:
       Promo_Model += R[i][t] <= X[i][t] 
        
for t in PHorizon:
    Promo_Model += lpSum(X[i][t] for i in Products) == 1
    
for i in Products:
    Promo_Model += lpSum(X[i][t] for t in PHorizon) == 3

#PROMO RATIO
for i in Products:
    for t in PHorizon:
        Promo_Model += F[i][t] == R[i][t]*Demand[i][t]            
    
#TOTAL PAID QUANTITY CAPACITY
for i in Products:
    for t in PHorizon:
        Promo_Model += P[i][t] <= Total_PaidQty_Limit
    
#TOTAL FREE QUANTITY CAPACITY
for i in Products:
    for t in PHorizon:
        Promo_Model += F[i][t] <= Total_FreeQty_Limit

In [116]:
Promo_Model.solve()
# The status of the solution is printed to the screen
print("Status:", LpStatus[Promo_Model.status])

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/baris/anaconda3/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/11/n43tq6bj255drztgc6ydfmdc0000gn/T/360495e2bda946d490608302b4de111a-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/11/n43tq6bj255drztgc6ydfmdc0000gn/T/360495e2bda946d490608302b4de111a-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 331 COLUMNS
At line 1142 RHS
At line 1469 BOUNDS
At line 1590 ENDATA
Problem MODEL has 326 rows, 120 columns and 600 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.00 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00

Status: Infeasible


In [114]:
# OUTPUT

# OPTIMIZED OBJECTIVE FUNCTION
print("Optimal Total Paid Quantity = ", pulp.value(Promo_Model.objective),"\n")

# PRINTS VARIABLES AND OPTIMAL VALUES
for v in Promo_Model.variables():
    print(v.name, "=", v.varValue)

Optimal Total Paid Quantity =  60000.0 

Free_Quantity_0_0 = 0.0
Free_Quantity_0_1 = 0.0
Free_Quantity_0_2 = 0.0
Free_Quantity_0_3 = 0.0
Free_Quantity_0_4 = 0.0
Free_Quantity_0_5 = 6848.0
Free_Quantity_0_6 = 0.0
Free_Quantity_0_7 = 0.0
Free_Quantity_0_8 = 0.0
Free_Quantity_0_9 = 6830.0
Free_Quantity_1_0 = 0.0
Free_Quantity_1_1 = 0.0
Free_Quantity_1_2 = 0.0
Free_Quantity_1_3 = 6377.0
Free_Quantity_1_4 = 0.0
Free_Quantity_1_5 = 0.0
Free_Quantity_1_6 = 0.0
Free_Quantity_1_7 = 0.0
Free_Quantity_1_8 = 0.0
Free_Quantity_1_9 = 0.0
Free_Quantity_2_0 = 7604.0
Free_Quantity_2_1 = 0.0
Free_Quantity_2_2 = 0.0
Free_Quantity_2_3 = 0.0
Free_Quantity_2_4 = 7765.0
Free_Quantity_2_5 = 7422.0
Free_Quantity_2_6 = 0.0
Free_Quantity_2_7 = 0.0
Free_Quantity_2_8 = 0.0
Free_Quantity_2_9 = 0.0
IsPromoted_0_0 = 0.0
IsPromoted_0_1 = 0.0
IsPromoted_0_2 = 0.0
IsPromoted_0_3 = 0.0
IsPromoted_0_4 = 0.0
IsPromoted_0_5 = 0.0
IsPromoted_0_6 = 0.0
IsPromoted_0_7 = 0.0
IsPromoted_0_8 = 0.0
IsPromoted_0_9 = 0.0
IsPromoted_